In [ ]:
import validmind as vm

vm.init(
  api_host = "https://api.dev.vm.validmind.ai/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

In [ ]:
import xgboost as xgb

%matplotlib inline

In [ ]:
from validmind.datasets.classification import customer_churn as demo_dataset

print(f"Loaded demo dataset with: \n\n\t• Target column: '{demo_dataset.target_column}' \n\t• Class labels: {demo_dataset.class_labels}")

raw_df = demo_dataset.load_data()
raw_df.head()

In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(raw_df)

x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

In [ ]:
type(test_df['Exited'])

In [ ]:
feature_columns = [col for col in test_df.columns if col != demo_dataset.target_column]
feature_columns

In [ ]:
# Compute predictive probabilities for the test dataset
# Here, we only use the probabilities for the positive class (class 1)
predictive_probabilities = model.predict_proba(test_df.drop(demo_dataset.target_column, axis=1))[:, 1]

# Add the predictive probabilities as a new column to the test dataframe
test_df['PredictiveProbabilities'] = predictive_probabilities

# Add the predictions from the predictive probabilities as a new column to the test dataframe
test_df['Predictions'] = (predictive_probabilities > 0.5).astype(int)

# Display the first few rows of the updated dataframe to verify
test_df.head()

In [ ]:
import validmind as vm

vm_test_ds = vm.init_dataset(
    test_df, 
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns,
    extra_columns={"prediction_column": "Predictions" })

In [ ]:
vm_test_ds.prediction_column

In [ ]:
feature_columns

In [ ]:
vm_test_ds.feature_columns

In [ ]:
vm_test_ds.y

In [ ]:
vm_test_ds._df

In [ ]:
vm_model = vm.init_model(
    model,
    input_id="my_model")

In [ ]:
import validmind as vm

metric_id = "validmind.metrics.sklearn.classification.F1"

inputs = {
    "model": vm_model,
    "dataset": vm_test_ds
}

params = {"average": "macro"}

result = vm.run_metric(

    metric_id=metric_id, 
    inputs=inputs,
    params=params
    
)

In [ ]:
import validmind as vm

metric_id = "validmind.metrics.sklearn.classification.F1"

inputs = {
    "model": None,
    "dataset": vm_test_ds
}

params = {"average": "macro"}

result = vm.run_metric(

    metric_id=metric_id, 
    inputs=inputs,
    params=params
    
)

In [ ]:
result.metric.value

In [ ]:
result.metric.summary

In [ ]:
import validmind as vm

metric_id = "validmind.metrics.sklearn.classification.Precision"

inputs = {"dataset": vm_test_ds}

result = vm.run_metric(

    metric_id=metric_id, 
    inputs=inputs,
    
)

In [ ]:
import validmind as vm

metric_id = "validmind.metrics.sklearn.classification.Precision"

inputs = {"dataset": vm_test_ds}

result = vm.run_metric(

    metric_id=metric_id, 
    inputs=inputs,
    
)

In [ ]:
import validmind as vm

metric_id = "validmind.metrics.sklearn.classification.Recall"

inputs = {"dataset": vm_test_ds}

result = vm.run_metric(

    metric_id=metric_id, 
    inputs=inputs,
    
)

In [ ]:
import validmind as vm

metric_id = "validmind.metrics.sklearn.classification.Accuracy"

inputs = {"dataset": vm_test_ds}

result = vm.run_metric(

    metric_id=metric_id, 
    inputs=inputs,
    
)
result.metric.value

In [ ]:
import validmind as vm

metric_id = "validmind.metrics.sklearn.classification.Accuracy"

params = {"normalize": False}

inputs = {
    "model": vm_model,
    "dataset": vm_test_ds
}

result = vm.run_metric(

    metric_id=metric_id, 
    inputs=inputs,
    params=params
    
)
result.metric.value

In [ ]:
import validmind as vm

metric_id = "validmind.metrics.sklearn.classification.ROC_AUC"

params = {
    "average": "macro",
    "multi_class": "raise"
}

inputs = {
    "model": None,
    "dataset": vm_test_ds
}

result = vm.run_metric(

    metric_id=metric_id, 
    inputs=inputs,
    params=params
    
)
result.metric.value